<h1><center>
Computational Notebook of "A sensitivity analysis of a regression model of ocean temperature"
</center></h1>

## Context


Can data-driven models for weather and climate predictions learn the underlying physics of the system against which they are trained? Or are they simply capable of identifying statistical patterns without any clear link to the underlying physics? In *["A sensitivity analysis of a regression model of ocean temperature"](https://www.cambridge.org/core/journals/environmental-data-science/article/sensitivity-analysis-of-a-regression-model-of-ocean-temperature/089EA5B347F262CEE0B4BDFFBA4E6FF5)*, Furner, R. at al. (2022) run a sensitivity analysis of a regression-based model of ocean temperature, trained against simulations from a 3D ocean model setup, demostrating that regression models are capable of learning much of the physics of the underlying system, such as currents and density.


### Purpose
This notebook aims to complement the science and methodological development embedded within the original paper, using an open infrastructure that allows users to combine interactive code with text and graphical objects, translating research outputs into findable, accessible, interoperable and reusable outputs and code.

### Description
*Describe the main features of the dataset/sensor, preprocessing, modelling or postprocessing routine*

#### Data
The training and validation data derive from running the Massachusetts Institute of Technology general circulation model ([MITgcm](https://zenodo.org/record/4672260#.ZFgb_S-B2pY)) —a physically based model capable of simulating the ocean or the atmosphere dynamics due to isomorphisms in the governing equations. This research project relies on a simple 2° sector configuration that captures the fundamental dynamics of the ocean, including a realistic overturning circulation. This configuration features a single ocean basin, with:
+ limited topography, 
+ simplified coastlines, and 
+ constant idealized forcing. 

The domain runs from 60°S to 60° N, and is just over 20° wide in longitude. The domain is bounded by land along its northern and southern edges, and a strip of land runs along the eastern/western boundary from 60° N to 40° S.  Below this, in the southernmost 20°, the simulator has a periodic boundary condition, allowing flow that exits to the east and west to return to the domain at the western and eastern boundary. The domain has flat-bottom bathymetry of 5,000 m over most of the domain, with a 2° region of 2,500-m depth at the southernmost 20° of the eastern edge.

The simulator has:

+ 42 depth levels, with the surface layer being the thinnest at 10 m, and the bottom 10 levels being the maximum at 250 m. 
+ 11 cells in the longitudinal direction, and
+ 78 cells in the latitudinal direction.

The simulator has a 12-hr time step, with fields output daily. We focus on daily-mean outputs.

### Highlights

*Provide 3-5 bullet points that convey the use case’s core procedures. Each bullet point must have a maximum of 85 characters, including spaces.*
* Highlight 1
* Highlight 2

### Contributions
*Please indicate key contributions. Notebook authors (mandatory). Optional subheadings, used in published EDS book notebooks, are Dataset originator/creator, Dataset authors, Dataset documentation, Modelling codebase, Modelling publications, Modelling funding and Pipeline documentation. Feel free to add any of them or propose new ones to attribute relevant research contributions.*

#### Notebook
* Author (role), Affiliation, GitHub alias

:::{note}
*Optional: add credits or acknowledgements to data and/or model providers or authors of code snippets*
:::

## Load libraries
List libraries according to their role e.g. system/files manipulation i.e. os (first), data handling i.e. numpy, xarray (second), visualisation i.e. holoviews (third), etc. The cell below contains two libraries, `os` and `warning` which are common among the Python Jupyter notebooks. Don't remove them.*

In [1]:
import os
import gc
import sys
import pickle
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import xarray as xr
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

## Set project structure
*The cell below creates a separate folder to save the notebook outputs. This facilitates the reader to inspect inputs/outputs stored within a defined destination folder. Don't remove the lines below.*

In [ ]:
notebook_folder = './notebook'
if not os.path.exists(notebook_folder):
    os.makedirs(notebook_folder)

## Load data/model
*Load full dataset or model from original or mirror sources. If the license of the dataset/model permits, we suggest creating sample data (preprocessed) or model mirror for the notebook stored in a data repository e.g. Zenodo.*

In [ ]:
# Import data from Zenode?

In [2]:
# Read in the data
ds = xr.open_dataset("../data/raw/cat_tave.nc")

In [3]:
ds

<xarray.Dataset>
Dimensions:      (T: 18000, Y: 78, X: 11, Z: 42, Xp1: 12, Yp1: 79, Zl: 42,
                  Zld000042: 42, Zmd000042: 42)
Coordinates:
  * T            (T) float64 1.555e+09 1.555e+09 1.555e+09 ... 3.11e+09 3.11e+09
  * X            (X) float64 1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0 17.0 19.0 21.0
  * Xp1          (Xp1) float64 0.0 2.0 4.0 6.0 8.0 ... 14.0 16.0 18.0 20.0 22.0
  * Y            (Y) float64 -59.5 -58.48 -57.44 -56.36 ... 58.48 59.5 60.5 61.5
  * Yp1          (Yp1) float64 -60.0 -59.0 -57.97 -56.91 ... 59.0 60.0 61.0 62.0
  * Z            (Z) float64 -5.0 -15.05 -25.25 ... -4.625e+03 -4.875e+03
  * Zl           (Zl) float64 0.0 -10.0 -20.1 ... -4.25e+03 -4.5e+03 -4.75e+03
Dimensions without coordinates: Zld000042, Zmd000042
Data variables: (12/23)
    ETAtave      (T, Y, X) float32 ...
    Stave        (T, Z, Y, X) float32 ...
    Ttave        (T, Z, Y, X) float32 ...
    uVeltave     (T, Z, Y, Xp1) float32 ...
    vVeltave     (T, Z, Yp1, X) float32 ...
    wVeltave     (T, Zl, Y, X) float32 ...
    ...           ...
    TOTTTEND     (T, Zmd000042, Y, X) float32 ...
    UVELTH       (T, Zmd000042, Y, Xp1) float32 ...
    VVELTH       (T, Zmd000042, Yp1, X) float32 ...
    WVELTH       (T, Zld000042, Y, X) float32 ...
    diag_levels  (Zmd000042) float64 ...
    Mask         (Z, Y, X) int32 ...
Attributes: (12/26)
    the_run_name:               2deg
    MITgcm_version:             checkpoint67r
    build_user:                 racfur
    build_host:                 bslws05
    build_date:                 Tue 24 Nov 16:24:06 GMT 2020
    MITgcm_URL:                 http://mitgcm.org
    ...                         ...
    Ny:                         78
    Nr:                         42
    history:                    Thu Nov 26 14:27:31 2020: ncks -A cat_diag_ta...
    NCO:                        netCDF Operators version 4.7.5 (Homepage = ht...
    nco_openmp_thread_number:   1
    history_of_appended_files:  Thu Nov 26 14:27:31 2020: Appended file cat_d...

In [6]:
da_T = ds['Ttave'].values

In [7]:
da_T.shape

(18000, 42, 78, 11)

## Model training

## Visualisation
*Create a visual narrative of the dataset! We suggest exploring libraries suited for interactive plotting e.g. python (Holoviews, Panel, Bokeh).*

## Summary
*Provide 3-5 bullet points summarising the main aspects of the dataset and tools covered in the notebook.*

* Sentence 1 e.g. `tool-name` to perform...
* Sentence 2 e.g. `tool-name` to perform...

## Additional information
**Dataset**: Type here details of dataset(s) version.

**Codebase**: Type here details of codebase version (only for notebooks categorised under modelling/preprocesing/post-processing themes).

**License**: The code in this notebook is licensed under the MIT License. The Environmental Data Science book is licensed under the Creative Commons by Attribution 4.0 license. See further details [here](https://github.com/alan-turing-institute/environmental-ds-book/blob/master/LICENSE.md).

**Contact**: If you have any suggestion or report an issue with this notebook, feel free to [create an issue](https://github.com/alan-turing-institute/environmental-ds-book/issues/new/choose) or send a direct message to [environmental.ds.book@gmail.com](mailto:environmental.ds.book@gmail.com).

In [ ]:
from datetime import date
print(f'Last tested: {date.today()}')